In [3]:
import numpy as np
import matplotlib.pyplot as plt
from plant import plant

In [12]:
import pandas as pd                        #importing necessary libraries to read the excel file
import numpy as np

#from Standardize import *
df = pd.read_excel('New_helicopter.xlsx')

print (df.describe())

n = 500
z = df.sample(n)                                 #randomly sampling from the dataframe
x = z.iloc[:,0:7]                                #first 7 columns are inputs
x = np.array(x).astype(np.float32)
mean = np.mean(x, axis=0)                        #columnwise mean and
sd = np.std(x, axis=0)                           #standard deviation
mean = mean.reshape(7, 1)
sd = sd.reshape(7, 1)
m = mean[0:5]
s = sd[0:5]

def standardise(Z, t='x'):                      #to standardise the data(basically pre-processing)
    if t == 'y':
        Z = (Z - m)/s
    else:
        Z = (Z - mean)/sd
    return Z


def reverse_standardise(Z, t='x'):              #to reverse-standardise the data
    if t == 'y':
        Z = (Z * s) + m
    else:
        Z = (Z * sd) + mean
    return Z

            Inputs  Unnamed: 1  Unnamed: 2  Unnamed: 3   Unnamed: 4  \
count   2177.00000        2177     2177.00   2177.0000  2177.000000   
unique  1672.00000        1188     1453.00   1336.0000  1672.000000   
top      154.61172           2       -2.85      0.0175    -1.976539   
freq       2.00000         349      220.00    337.0000     2.000000   

         Unnamed: 5   Unnamed: 6      Outputs  Unnamed: 8  Unnamed: 9  \
count   2177.000000  2177.000000  2177.000000        2177     2177.00   
unique  1672.000000  1672.000000  1672.000000        1187     1452.00   
top      188.476424  -170.848222   154.610087           2       -2.85   
freq       2.000000     2.000000     2.000000         350      221.00   

        Unnamed: 10  Unnamed: 11  
count     2177.0000  2177.000000  
unique    1336.0000  1672.000000  
top          0.0175    -1.970298  
freq       337.0000     2.000000  


In [25]:
def plot_errors(Errors):
    
    """Used for error plots."""
    
    plt.xlabel("Timesteps", fontsize=50)
    plt.ylabel("Error", fontsize=50)
    plt.axhline(y=0, color='r', linestyle='-', lw=10)
    plt.plot(Errors, linewidth=5, color='k')
    plt.tick_params(axis='both', which='major', labelsize=45)

In [26]:
def plot_outputs(P, j, desired_value, print_values=False):
    
    """Plots a particular output of the plant."""
    
    P = np.array(P)
    P = P.flatten().reshape(P.shape[0],5)
    if print_values: print(P[:,  j])    #for printing the values
    plt.xlabel("Timesteps", fontsize=25)
    plt.ylabel("Value", fontsize=25)
    plt.axhline(y=desired_value, color='r', linestyle='-', lw=4)
    plt.plot(P[:,  j], linewidth=2.5)
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.title("Plant Output {}".format(j + 1), fontsize=30, loc='center')

In [36]:
def RTRN(W, Y, U, print_weights=True):        
    
    """Common model for both RTRNs; 
    takes standardised input and provides standardised output."""
    
    W1 = W[:, :5]                
    W2 = W[:, 5:] 
    if print_weights:
        print("W:\n")
        print(W, "\n")
    Z1 = np.dot(W1, Y)
    Z2 = np.dot(W2, U)
    Z = Z1 + Z2
    #print("W1 ", W1.shape, "W2 ", W2.shape, "Y ", Y.shape, "U ", U.shape, "Z1 ", Z1.shape, 
    #      "Z2 ", Z2.shape, "Z ", Z.shape)
    output = np.tanh(Z)          #for tanh activation
    return output

def c_inverse(W, x, delta):             
    
    """Calculates a quantity used in the update_weights function."""
    
    ji = np.dot(W, x) 
    ji_ = 1 - np.tanh(ji)**2     #derivative of  tanh activation
    ji_ = ji_.reshape(-1,)
    c = np.diag(ji_)             
    delta = np.eye(5) * delta
    c += delta                   #small delta is used to prevent singular matrices
    c_inv = np.linalg.inv(c)        
    return c_inv

def U_control(W, W_star, Y_d, Y_next, U):            
    
    """Control law; approximates the system output to the desired output.
    Updates only the 2 control inputs U1 and U2."""
    
    W1 = W[:, :5]                
    W2 = W[:, 5:] 
    W1_star = W_star[:, :5]
    W2_star = W_star[:, 5:]
    W2_g = np.linalg.pinv(W2)
    Z = np.dot(W1_star, Y_d) - np.dot(W1, Y_next) + np.dot(W2_star, U)
    U_star = np.dot(W2_g, Z)
    return U_star


def update_weights(W_old, Y, D, Y_next, delta):                
    
    """Learning algorithm of both the RTRNs."""
    #print('update weights')
    
    c_inv = c_inverse(W_old, Y, delta)                         #small delta is used 
    e1 = (D - Y_next)                                          
    e2 = np.linalg.inv(np.dot(Y.T, Y))
    e2 = np.dot(e2, Y.T)
    e = np.dot(e1, e2)
    W_new = W_old + np.dot(c_inv, e)
    return W_new

In [37]:
#TRAINING CELL
#THE RESULTS ARE SENSITIVE TO INITIAL WEIGHTS

np.random.seed(39)

Plant = plant()                       #creating an object of the plant class

z = df.sample(1)
timesteps = 1000

W = np.random.random((5,7))           #initial weights for RTRN1
W_star = np.random.random((5,7))      #initial weights for RTRN2

x = np.array(z.iloc[0 ,:7])
x = pd.to_numeric(x)
x = x.reshape(7, 1)
X = standardise(x)
RTRN1_input = RTRN2_input = X

Plant_outputs = []
Errors1 = []
Errors2 = []

for t in range(timesteps):
    
    
    Y1 = RTRN1_input[:5]
    Y2 = RTRN2_input[:5]
    U = RTRN1_input[5:]
    x = reverse_standardise(RTRN1_input)
    itr = 0
    
    
    ####################################
    while True:         #RTRN1 do-while                             
        itr += 1
        
        RTRN1_output = RTRN(W, Y1, U, print_weights=False)                            
        x = x.reshape(-1,)
        
        Plant_output = Plant.Predict(x)                            
        Plant_output = Plant_output.reshape(5,1)
        Plant_output_std = standardise(Plant_output, 'y')                       
        
        W = update_weights(W, RTRN1_input, Plant_output_std, RTRN1_output, 0.2)       
        
        e = np.sum(np.square(Plant_output_std - RTRN1_output), axis=0) / 2
        if(e < 1):
            break
    ####################################    
    
    char = "RTRN1"
    print("Timestep {0} has {1} {2} iteration(s)".format(t+1, itr, char)) 
    
    itr = 0    
    
    
    ####################################
    while True:         #RTRN2 do-while
        itr += 1
        
        RTRN2_output = RTRN(W_star, Y2, U, print_weights=False)   
        RTRN2_output_rev_std = reverse_standardise(RTRN2_output, 'y')
        
        Desired_output = np.array([140, 0.5, RTRN2_output_rev_std[2][0], 
                                   RTRN2_output_rev_std[3][0], RTRN2_output_rev_std[4][0]])        
        Desired_output = Desired_output.reshape(5,1)
        Desired_output_std = standardise(Desired_output, 'y')
        
        W_star = update_weights(W_star, RTRN2_input, Desired_output_std, RTRN2_output, 0.2)        
        
        e = np.sum(np.square(Desired_output_std - RTRN2_output), axis=0) / 2
        if(e < 0.1):
            break
    ####################################        

    char = "RTRN2"
    print("Timestep {0} has {1} {2} iteration(s)".format(t+1, itr, char)) 
    
    
    U = U_control(W, W_star, RTRN2_output, RTRN1_output, U)
    
    RTRN1_input[:5] = RTRN1_output
    RTRN1_input[5:] = U
    RTRN2_input[:5] = RTRN2_output
    RTRN2_input[5:] = U
    
    Desired = [140, 0.5]
    
    Plant_outputs.append(Plant_output)
    Errors1.append(Desired[0] - Plant_output[0])
    Errors2.append(Desired[1] - Plant_output[1])
    
    print("\n END OF TIMESTEP {}".format(t+1))
    print(40*"#","\n\n")

Timestep 1 has 3 RTRN1 iteration(s)


KeyboardInterrupt: 

In [ ]:
#PLOTTING OUTPUTS
plt.figure(figsize=(30, 10))
plt.subplot(1, 2, 1)
plot_outputs(Plant_outputs, 0, 140)
plt.subplot(1, 2, 2)
plot_outputs(Plant_outputs, 1, 0.5)

In [ ]:
#PLOTTING ERRORS
Errors1 = np.array(Errors1)
Errors2 = np.array(Errors2)
plt.figure(figsize=(80, 40))
plt.subplot(2, 1, 1)
plot_errors(Errors1)
plt.title("Error Plots", fontsize=140, loc='center')
plt.subplot(2, 1, 2)
plot_errors(Errors2)
plt.show()